In [19]:
import random
import math

In [13]:
class AutoTaxi:
    is_ai=True

    def __init__(self, agent_class, environ, start_loc, end_loc):
        self.environ = environ

        # Invariant: x,y is inside environ bounds for both start_loc & end_loc
        self.start_loc = start_loc
        self.end_loc = end_loc
        
        self.position = self.start_loc

        self.agent = agent_class(self)
        
    def move_next(self):
        action = self.agent.act( self.environ )
        
        # Now actuator, after getting action from the agent
        self._move(direction=action)
        
    # Invariant: direction is valid (ie. not cross border)
    def _move(self, direction):
        if direction[0] == 0 and direction[1] == 0:
            print("[", self.agent, "] Not Moving")
            return

        print("[",self.agent,"] Taxi moving", end=" ")
        if direction[0] == -1:
            print("Left", end=" ")
        elif direction[0] == 1:
            print("Right", end=" ")
        if direction[1] == -1:
            print("Up", end=" ")
        elif direction[1] == 1:
            print("Down", end=" ")
        print()

        new_x = self.position[0] + direction[0]
        new_y = self.position[1] + direction[1]

        self.position = [ new_x, new_y ]


In [14]:
# Model-based reflex agent
class ModelBasedReflexAgent:
    def __init__(self,parent_taxi):
        self.taxi = parent_taxi
        # At a time, it keeps notice of what nearby obstacles are present
        # In this simple model, I am considering the obstacles to be the model data, as depending on what area are movable it will move (state.obstacle is for next move itself)
        self.state = {
            "end_loc": self.taxi.end_loc,
            "obstacles": []
        }

    def _state_update(self):
        curr_pos = self.taxi.position
        environ = self.taxi.environ
        obstacles = []

        # Taxi can change it's destination at runtime, so we keep model's state coherent
        self.state["end_loc"] = self.taxi.end_loc
        
        dirs = [[-1,-1],[-1,0],[-1,1],[0,-1],[0,1],[1,-1],[1,0],[1,1]]
        for dir in dirs:
            x = curr_pos[0] + dir[0]
            y = curr_pos[1] + dir[1]
            if environ.matrix[x][y].is_blocked:
                obstacles.append([x,y])

        self.state["obstacles"] = obstacles

    def act(self,percept):
        # The state <- Update-State() line in ppt slide, as per the simple model, that NEXT MOVE DEPENDS ON WHAT MOVABLE LOCATION IS
        self._state_update()

        pos = self.taxi.position

        if pos == self.state["end_loc"]:
            print("[",self,"] ALREADY REACHED DESTINATION...")
            return [0,0]

        movable_directions = []
        
        dirs = [[-1,-1],[-1,0],[-1,1],[0,-1],[0,1],[1,-1],[1,0],[1,1]]
        
        for dir in dirs:
            x = pos[0] + dir[0]
            y = pos[1] + dir[1]

            # TODO
            if x < 0 or x >= len(self.taxi.environ.matrix) or y < 0 or y >= len(self.taxi.environ.matrix):
                continue

            if [x,y] not in self.state["obstacles"]:
                movable_directions.append(dir)

        # HERE IS WHERE a Goal-Based Reflex agent will perform better
        # Instead of randomly chosing it will chose the location that gets us near to goal
        return random.choice(movable_directions)

In [15]:
# Goal-based reflex agent

# Sir my understanding is that Goal based agent is kind of Model-based + chose best move among possible moves/actions
# so assuming that, it is mostly same, except some additions in Agent.act function
class GoalBasedReflexAgent:
    def __init__(self,parent_taxi):
        self.taxi = parent_taxi
        self.state = {
            "goal": self.taxi.end_loc,
            "obstacles": []
        }

    def _state_update(self):
        curr_pos = self.taxi.position
        environ = self.taxi.environ
        obstacles = []

        # Taxi can change it's destination at runtime, so we keep model's state coherent
        self.state["goal"] = self.taxi.end_loc
        
        dirs = [[-1,-1],[-1,0],[-1,1],[0,-1],[0,1],[1,-1],[1,0],[1,1]]
        for dir in dirs:
            x = curr_pos[0] + dir[0]
            y = curr_pos[1] + dir[1]
            if environ.matrix[x][y].is_blocked:
                obstacles.append([x,y])

        self.state["obstacles"] = obstacles

    def act(self,percept):
        # The state <- Update-State() line in ppt slide, as per the simple model, that NEXT MOVE DEPENDS ON WHAT MOVABLE LOCATION IS
        self._state_update()

        movable_directions = []
        
        dirs = [[-1,-1],[-1,0],[-1,1],[0,-1],[0,1],[1,-1],[1,0],[1,1]]
        pos = self.taxi.position
        
        if pos == self.state["goal"]:
            print("[",self,"] ALREADY REACHED DESTINATION...")
            return [0,0]

        for dir in dirs:
            x = pos[0] + dir[0]
            y = pos[1] + dir[1]
            if [x,y] not in self.state["obstacles"]:
                movable_directions.append(dir)

        # Instead of randomly chosing it will chose the location that gets us near to goal

        # Chose the direction that will get us closer to goal
        better_direction = [0,0]
        goal = self.state["goal"]
        for dir in movable_directions:
            x = pos[0] + dir[0]
            y = pos[1] + dir[1]
            nearer_x = pos[0] + better_direction[0]
            nearer_y = pos[1] + better_direction[1]
            
            # https://stackoverflow.com/questions/5228383/how-do-i-find-the-distance-between-two-points#5228392
            nearest_dist = math.hypot( goal[1] - nearer_y, goal[0] - nearer_x )
            new_dist = math.hypot( goal[1] - y, goal[0] - x )

            if new_dist <= nearest_dist:
                better_direction = dir

        return better_direction

In [16]:
AutoTaxi(
        agent_class=ModelBasedReflexAgent,
        environ=[],
        start_loc=[5,4],  # this can be any random position inside the world
        end_loc=[15,17]
    )

In [18]:
AutoTaxi(
        agent_class=GoalBasedReflexAgent,
        environ=[],
        start_loc=[2,9],
        end_loc=[17,1]
    )